In [1]:
# initialize project:
using Pkg; Pkg.activate("./MyProject");;
include("libs/local_clifford_group.jl"); include("libs/cnc.jl"); include("libs/utils.jl");

  Activating project at `~/GitHub/LocalLambda/MyProject`


In [2]:
using JuMP

using JuMP
using GLPK
using LinearAlgebra

Generate Clifford group and local Clifford group

In [ ]:
# 3-qubit local Clifford group:
LC2 = LocalCliffordGroup(2); local_cl2 = LC2.LocalCliffGroup; PS2 = PauliString(2);

In [ ]:
# 3-qubit local Clifford group:
LC3 = LocalCliffordGroup(3); local_cl3 = LC3.LocalCliffGroup; PS3 = PauliString(3);

In [ ]:
# 2-qubit Clifford group
C2 = CliffordGroup(2); cl2 = C2.CliffGroup;

In [ ]:
# 3-qubit Clifford group
C3 = CliffordGroup(3); cl3 = C3.CliffGroup;

In [ ]:
PS2 = PauliString(2);
PS3 = PauliString(3);

In [ ]:
fieldnames(PauliString)

# magic states

In [ ]:
# Define magic state
T = Vector{Real}([1,1/sqrt(2),1/sqrt(2),0]);
T2 = Vector{Float64}(kron(T,T));;
T3 = Vector{Float64}(kron(kron(T,T),T));;
T4 = Vector{Float64}(kron(T3,T));;
T5 = Vector{Float64}(kron(T4,T));;
T6 = Vector{Float64}(kron(T5,T));;
T7 = Vector{Float64}(kron(T6,T));;
T8 = Vector{Float64}(kron(T7,T));;

In [ ]:
T3_L1 = sum(Vector{Real}([abs(x) for x in T3]))

In [ ]:
CNC1_L1 = sum(Vector{Real}([abs(x) for x in CNC1[:,1]]))

In [ ]:
CNC2_L1 = sum(Vector{Real}([abs(x) for x in CNC2[:,1]]))

In [ ]:
CNC3_L1 = sum(Vector{Real}([abs(x) for x in CNC3[:,1]]))

# one-qubit phase space

In [ ]:
cnc_sets_1_1 = generate_all_cncs(1,[1]);
cnc_1_1 = generate_cnc_vertices(1,cnc_sets_1_1);

In [ ]:
cnc_1_1

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(v1)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(v1) * x .== T)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
c1 = cnc_1_1[:,[3,4,6]]

In [ ]:
s1 = Matrix{Rational{Int64}}([1 1 1; 1 0 -1; 0 1 0; 0 0 0])

In [ ]:
S1 = Matrix{Rational{Int64}}([1 1 1 1 1; 1 0 -1 0 0; 0 1 0 -1 0; 0 0 0 0 1])

In [ ]:
c2 = Matrix{Rational{Int64}}(undef,16,0)
for i in 1:size(c1)[2]
    for j in 1:size(S1)[2]
        v = kron(c1[:,i],S1[:,j])
        c2 = hcat(c2,v)
    end
end

In [ ]:
c2

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c2)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c2) * x .== T2)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

# two-qubit cnc phase space:

In [ ]:
cnc_sets_2_2 = generate_all_cncs(2,[2]);
cnc_2_2 = generate_cnc_vertices(2,cnc_sets_2_2);

In [ ]:
cnc_sets_2_1 = generate_all_cncs(2,[1]);
cnc_2_1 = generate_cnc_vertices(2,cnc_sets_2_1);

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(cnc_2_1)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(cnc_2_1) * x .== T2)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

In [ ]:
cnc_2_1[:,non_zero_optimal_2_1]

In [ ]:
non_zero_optimal_2_1 = findall(x->x>(10^(-5)),x_value)

for x in non_zero_optimal_2_1
    display(pauli_basis_to_cnc(cnc_2_1[:,x],PS2))
end

In [ ]:
c3 = Matrix{Rational{Int64}}(undef,64,0)
for i in 1:size(cnc_2_1)[2]
    for j in 1:size(s1)[2]
        v = kron(cnc_2_1[:,i],S1[:,j])
        c3 = hcat(c3,v)
    end
end

In [ ]:
c3

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c3) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
c4 = Matrix{Rational{Int64}}(undef,64*4,0)
for i in 1:size(c3)[2]
    for j in 1:size(s1)[2]
        v = kron(c3[:,i],S1[:,j])
        c4 = hcat(c4,v)
    end
end

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c4) * x .== T4)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
c5 = Matrix{Rational{Int64}}(undef,64*4*4,0)
for i in 1:size(c4)[2]
    for j in 1:size(s1)[2]
        v = kron(c4[:,i],S1[:,j])
        c5 = hcat(c5,v)
    end
end

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c5)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c5) * x .== T5)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
c6 = Matrix{Rational{Int64}}(undef,64*4*4*4,0)
for i in 1:size(c5)[2]
    for j in 1:size(s1)[2]
        v = kron(c5[:,i],s1[:,j])
        c6 = hcat(c6,v)
    end
end

In [ ]:
s5 = Matrix{Rational{Int64}}(undef,4^5,0)
for i1 in 1:3
    for i2 in 1:3
        for i3 in 1:3
            for i4 in 1:3
                for i5 in 1:3
                    v = kron(s1[:,i5],kron(s1[:,i4],kron(s1[:,i3],kron(s1[:,i2],s1[:,i1]))));
                    s5 = hcat(s5,v)
                end
            end
        end
    end
end

In [ ]:
c7 = kron(cnc_2_1[:,non_zero_optimal_2_1],s5)

In [ ]:
c6

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c6)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c6) * x .== T6)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c7)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c7) * x .== T7)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
c8 = kron(kron(cnc_2_1[:,non_zero_optimal_2_1],s5),s1)

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(c8)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(c8) * x .== T8)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
using JLD2

@save "./data/x_value_8.jld" x_value
@save "./data/cnc_vectors_subset_8.jld" c8

In [ ]:
load("./data/cnc_vectors_subset_8.jld")

In [ ]:
x_value

In [ ]:
findall(x->abs(x)>10^-6,x_value)

# Three-qubit CNC phase space

In [ ]:
cncs1 = generate_all_cncs(3,[1]);
CNC1 = generate_cnc_vertices(3,cncs1);

In [ ]:
CNC1

In [ ]:
cncs2 = generate_all_cncs(3,[2]);
CNC2 = generate_cnc_vertices(3,cncs2);

In [ ]:
CNC2

In [ ]:
cncs3 = generate_all_cncs(3,[3]);
CNC3 = generate_cnc_vertices(3,cncs3);

In [ ]:
CNC3

In [ ]:
cncs = generate_all_cncs(3,[1,2,3]);
CNC = generate_cnc_vertices(3,cncs);

In [ ]:
CNC

In [ ]:
s3 = generate_all_cncs(3,[0]);
stab3 = generate_cnc_vertices(3,s3);

In [ ]:
for i in 1:3    
    cncs_i = generate_all_cncs(3,[i]);
    CNCi = generate_cnc_vertices(3,cncs_i);
    println(size(CNCi))
end

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(CNC1[:,non_zero_optimal_3_1])[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(CNC1[:,non_zero_optimal_3_1]) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

In [ ]:
x_abs = Vector{Real}([abs(x) for x in x_value])
#x_min = minimum(x_abs[findall(x->abs(x)>0,x_value)])
x_min = 10^(-15);;

In [ ]:
non_zero_optimal_3_1 = findall(x->x>x_min,x_abs)

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(CNC2)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(CNC2) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(CNC3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(CNC3) * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

# four-qubit cnc

In [ ]:
using JLD2

cnc_4_1_orbit = load("./data/cnc_orbit_4_1.jld")["cnc_orbit_4_1"]

In [ ]:
using JLD2
cnc_vectors_4_1 = load("./cnc_vectors_4_1.jld")["cnc_vectors_4_1"]

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(cnc_vectors_4_1)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(cnc_vectors_4_1) * x .== T4)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

# three-qubit LC phase space

In [ ]:
d = Vector{Int}([1 for i in 1:64]);
D = array_to_matrix(local_clifford_orbit_of_point(LC3,d)); 

In [ ]:
V = Matrix{Rational{Int64}}(hcat(D,CNC))

In [ ]:
# Define controlled Z gate:
for g in C3.NormalizerGates.Gens; println(g); end

In [ ]:
# Unitaries:
CX12 = cnot(1,2,3); CX13 = cnot(1,3,3); CX23 = cnot(2,3,3);
H1 = hadamard_j(1,3); H2 = hadamard_j(2,3); H3 = hadamard_j(3,3);

In [ ]:
cx12 = C3.GateGroupHom(CX12); cx13 = C3.GateGroupHom(CX13); cx23 = C3.GateGroupHom(CX23);
h1 = C3.GateGroupHom(H1); h2 = C3.GateGroupHom(H1); h3 = C3.GateGroupHom(H1); 
cz12 = h2*cx12*g.Inverse(h2); cz13 = h3*cx13*g.Inverse(h3); cz23 = h3*cx23*g.Inverse(h3);

In [ ]:
function clifford_gate_action_by_gens(gens::Vector{GapObj},CG::CliffordGroup,C::Vector)
    C = Vector{Real}(C)
    fdict = CG.IntActionDict
    bdict = CG.ActionIntDict
    G = CG.CliffGroup.Grp
    N = length(C);
    K = g.Set(jlg([]))

    gg = gens[1];
    for elem in gens[2:end]; gg = elem*gg; end;

    vals, V, map = grouping(C)

    K = Vector{Vector{Int}}([])
    for v in V
        k = Vector{Int}([]);
        for e in v
            x = e; y = fdict[e]; gy = g.OnPoints(jlg(y),gg); gx = bdict[gy];
            #println("$x, $y, $gy, $gx")
            push!(k,gx); 
        end
        push!(K,k)
    end

    return subset_to_vector(N,K,map)
end

These are magic graph states where the graphs are the line $L_3$ and the complete graph $K_3$.

In [ ]:
L3 = clifford_gate_action_by_gens([cz12,cz23],C3,T3);
K3 = clifford_gate_action_by_gens([cz12,cz23,cz13],C3,T3);

These are magic graph states where the initial magic state $|T\rangle^{\otimes 3}$ is acted on by a unitary $\bar H^U$, which applies $H$ to qubit $i$ if $i\in U$, where $U$ is a subset of the vertex set of $G$.

In [ ]:
L3 = clifford_gate_action_by_gens([cz12,cz23],C3,T3);
H1L3 = clifford_gate_action_by_gens([h1,cz12,cz23],C3,T3);
H2L3 = clifford_gate_action_by_gens([h2,cz12,cz23],C3,T3);
H3L3 = clifford_gate_action_by_gens([h3,cz12,cz23],C3,T3);
H12L3 = clifford_gate_action_by_gens([h1,h2,cz12,cz23],C3,T3);
H13L3 = clifford_gate_action_by_gens([h1,h3,cz12,cz23],C3,T3);
H23L3 = clifford_gate_action_by_gens([h2,h3,cz12,cz23],C3,T3);
H123L3 = clifford_gate_action_by_gens([h1,h2,h3,cz12,cz23],C3,T3);

In [ ]:
K3 = clifford_gate_action_by_gens([cz12,cz23,cz13],C3,T3);
H1K3 = clifford_gate_action_by_gens([h1,cz12,cz23,cz13],C3,T3);
H2K3 = clifford_gate_action_by_gens([h2,cz12,cz23,cz13],C3,T3);
H3K3 = clifford_gate_action_by_gens([h3,cz12,cz23,cz13],C3,T3);
H12K3 = clifford_gate_action_by_gens([h1,h2,cz12,cz23,cz13],C3,T3);
H13K3 = clifford_gate_action_by_gens([h1,h3,cz12,cz23,cz13],C3,T3);
H23K3 = clifford_gate_action_by_gens([h2,h3,cz12,cz23,cz13],C3,T3);
H123K3 = clifford_gate_action_by_gens([h1,h2,h3,cz12,cz23,cz13],C3,T3);

In [ ]:
using JuMP

using JuMP
using GLPK
using LinearAlgebra

## Deterministic phase space

In [ ]:
# Dimensions
d = size(D)[1]; N = size(D)[2];

#### $L_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_1 \cdot L_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H1L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{2}\cdot L_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H2L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{12}\cdot L_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H12L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{13}\cdot L_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H13L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{123}\cdot L_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H123L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $K_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{1}\cdot K_3$

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H1K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H12K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(D)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, Matrix{Rational{Int64}}(D) * x .== H123K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

## Deterministic plus CNC

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, V * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, V * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

## CNC

In [ ]:
using JuMP
using GLPK
using LinearAlgebra

# Dimensions
d = size(V)[1]; N = size(CNC)[2];

Check that we recover correct value for CNC polytope.

In [ ]:
# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, CNC * x .== T3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

#### LC phase space with deterministic vertices and tensored $2$-qubit vertices

In [ ]:
using Polymake
const pm = Polymake;

include("libs/symmetries.jl");

Z = Vector{Rational{Int64}}([1,0,0,1])
ZZ = kron(Z,Z);
ZZZ = kron(Z,ZZ);

# Local 
S2 = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC2,ZZ)));
#S3 = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC3,ZZZ)));

In [ ]:
LL2 = pm.polytope.Polytope(INEQUALITIES = transpose(S2))

In [ ]:
R2 = transpose(Matrix{Rational{Int64}}(representative_vertices(LL2)));
tR3 = kron(R2,Vector{Rational{Int64}}([1,1,1,1]));

In [ ]:
R3 = Matrix{Rational{Int64}}(undef,64,0)
for k in 1:size(R2)[2]
    Vk = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC3,tR3[:,k])))
    println(size(Vk))
    R3 = hcat(R3,Vk)
end

#### $K_3$

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

#### $H_1\cdot K_3$

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H1K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{12}\cdot K_3$

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H12K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $H_{123}\cdot K_3$

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H123K3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

#### $L_3$

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H1L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H12L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R3)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R3 * x .== H123L3)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [ ]:
obj_value

# $4$-qubit LC robustness

In [ ]:
# 3-qubit local Clifford group:
LC4 = LocalCliffordGroup(4); local_cl4 = LC4.LocalCliffGroup; PS4 = PauliString(4);

In [ ]:
tR4 = kron(R2,Vector{Rational{Int64}}([1 for i in 1:16]));

In [ ]:
T4 = kron(T,T3);

In [ ]:
# Unitaries:
CX12 = cnot(1,2,4); CX13 = cnot(1,3,4); CX23 = cnot(2,3,4); CX34 = cnot(3,4,4); CX24 = cnot(2,4,4); CX14 = cnot(1,4,4);
H1 = hadamard_j(1,4); H2 = hadamard_j(2,4); H3 = hadamard_j(3,4);  H4 = hadamard_j(4,4);

cx12 = C3.GateGroupHom(CX12); cx13 = C3.GateGroupHom(CX13); cx23 = C3.GateGroupHom(CX23);
h1 = C3.GateGroupHom(H1); h2 = C3.GateGroupHom(H1); h3 = C3.GateGroupHom(H1); 
cz12 = h2*cx12*g.Inverse(h2); cz13 = h3*cx13*g.Inverse(h3); cz23 = h3*cx23*g.Inverse(h3);

In [ ]:
R4 = Matrix{Rational{Int64}}(undef,4^4,0)
for k in 1:size(tR4)[2]
    Vk = Matrix{Rational{Int64}}(array_to_matrix(local_clifford_orbit_of_point(LC4,tR4[:,k])))
    println(size(Vk))
    R4 = hcat(R4,Vk)
end

In [ ]:
using Serialization

# Save the group object to a file
open("large_group.jld", "w") do io
    serialize(io, LC4)
end

In [ ]:
using JLD2

# Save the matrix to a file
@save "R4.jld2" R4

# LC Robustness for $n \leq 8$ qubits

In [86]:
include("./magic_cluster_robustness.jl"); include("./deterministic_generation.jl");;

  Activating project at `~/GitHub/LocalLambda/MyProject`
  Activating project at `~/GitHub/LocalLambda/MyProject`


In [3]:
pauli_dicts = [PauliString(n) for n in 1:8];

In [4]:
using JuMP
using GLPK
using LinearAlgebra

## Four-qubit LC robustness

In [87]:
R4 = generate_all_deterministic_operators(4,pauli_dicts[4])

256×4096 Matrix{Int64}:
 1   1   1   1   1   1   1   1   1   1  …   1   1   1   1   1   1   1   1   1
 1   1   1   1  -1  -1  -1  -1   1   1     -1   1   1   1   1  -1  -1  -1  -1
 1   1  -1  -1   1   1  -1  -1   1   1     -1   1   1  -1  -1   1   1  -1  -1
 1  -1   1  -1   1  -1   1  -1   1  -1     -1   1  -1   1  -1   1  -1   1  -1
 1   1   1   1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1  -1  -1  -1  -1   1   1  …   1  -1  -1  -1  -1   1   1   1   1
 1   1  -1  -1   1   1  -1  -1   1   1      1  -1  -1   1   1  -1  -1   1   1
 1  -1   1  -1   1  -1   1  -1   1  -1      1  -1   1  -1   1  -1   1  -1   1
 1   1   1   1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1  -1  -1  -1  -1   1   1      1  -1  -1  -1  -1   1   1   1   1
 ⋮                   ⋮                  ⋱               ⋮                   ⋮
 1  -1   1  -1   1  -1   1  -1   1  -1      1  -1   1  -1   1  -1   1  -1   1
 1   1   1   1   1   1   1   1   1   1  

### Symmetric graphs:

In [77]:
# four qubit magic state:
n = 4
Tn = t_state(n);

# line graph:
L4 = copy(Tn)
for i in 1:n-1
    L4 = cz_action_on_pauli_basis(i,i+1,L4,pauli_dicts[n])
end

C4 = copy(Tn)
for i in 1:n
    if i < n
        C4 = cz_action_on_pauli_basis(i,i+1,C4,pauli_dicts[n])
    else
        C4 = cz_action_on_pauli_basis(1,n,C4,pauli_dicts[n])
    end
end

In [78]:
using Combinatorics

# Generate unique pairs (combinations) without self-loops
pairs = [p for p in combinations(1:n, 2)]

K4 = copy(Tn)
for p in pairs
    K4 = cz_action_on_pauli_basis(p[1],p[2],K4,pauli_dicts[n])
end

### Less symmetric graphs:

In [79]:
# remove one edge: 
G1 = copy(Tn)
for p in pairs[2:end]
    G1 = cz_action_on_pauli_basis(p[1],p[2],G1,pauli_dicts[n])
end

In [80]:
# remove two edges: Star
G2 = copy(Tn)
for p in pairs[3:end]
    G2 = cz_action_on_pauli_basis(p[1],p[2],G2,pauli_dicts[n])
end

In [81]:
# remove three edges:
G3 = copy(Tn)
for p in pairs[1:3]
    G3 = cz_action_on_pauli_basis(p[1],p[2],G3,pauli_dicts[n])
end

In [84]:
four_qubit_connected_graph_states = [L4,C4,K4,G1,G2,G3];
four_qubit_connected_graph_states_strings = ["L4","C4","K4","G1","G2","G3"]

6-element Vector{String}:
 "L4"
 "C4"
 "K4"
 "G1"
 "G2"
 "G3"

### Computation of robustness

In [66]:
using JuMP, GLPK, HDF5

function robustness(input_array, target_state; threshold=1e-16, save=true, file_loc="./", file_name="result")

    # Create a model with GLPK as the solver
    model = Model(GLPK.Optimizer)
    N = size(input_array)[2]

    # Define the decision variables
    @variable(model, x[1:N])
    @variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

    # Objective: Minimize the sum of u (which represents |x|)
    @objective(model, Min, sum(u))

    # Constraints for the absolute values
    @constraint(model, [i=1:N], u[i] >= x[i])
    @constraint(model, [i=1:N], u[i] >= -x[i])

    # Equality constraint: input_array * x = target_state
    @constraint(model, input_array * x .== target_state)

    # Solve the model
    optimize!(model)

    # Check the solver status
    if termination_status(model) != MOI.OPTIMAL
        error("Optimization failed with status: ", termination_status(model))
    end

    # Get the results
    x_value = value.(x)
    obj_value = objective_value(model)

    # Print the results
    println("Optimal objective value (min ||x||_1): ", obj_value)
    println("Optimal value of x: ", x_value)

    # Determine significant terms
    significant_terms = findall(x -> abs(x) > threshold, x_value)

    # Define dummy R4 if missing
    R4 = input_array  # Placeholder since R4 wasn't defined

    # Extract relevant terms
    decomposition_array = vcat(transpose(x_value[significant_terms]), input_array[:, significant_terms])

    if save
        # Save to HDF5 file
        h5_file = file_loc * file_name * ".h5"
        h5open(h5_file, "w") do file
            file[file_name] = decomposition_array
        end

        println("Optimal decomposition saved to $h5_file")
    end
end


robustness (generic function with 1 method)

In [88]:
# generate and save optimal decompositions:
for i in 1:6
    # initialize state
    graph_state = four_qubit_connected_graph_states[i]
    # get name
    graph_name = four_qubit_connected_graph_states_strings[i]
    # compute robustness and save
    robustness(R4, graph_state; threshold=1e-16, save=true, file_loc="./", file_name="deterministic_4_"*graph_name)
end

Optimal objective value (min ||x||_1): 2.164213562373095
Optimal value of x: [0.020336447899498157, 0.0021398922000010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9322546010351055e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03155978382878965, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.343332832087619e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005967569219566943, 0.0, 0.0, 0.0, 0.0, 0.0, 6.231255001236781e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00031503667232533325, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [51]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R4 * x .== L4)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 2.164213562373095
Optimal value of x: [0.020336447899498157, 0.0021398922000010925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9322546010351055e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03155978382878965, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.343332832087619e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005967569219566943, 0.0, 0.0, 0.0, 0.0, 0.0, 6.231255001236781e-18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00031503667232533325, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [62]:
L4_decomposition = vcat(transpose(x_value[findall(x->abs(x)>10^(-16),x_value)]),R4[:,findall(x->abs(x)>10^(-16),x_value)])

257×256 Matrix{Float64}:
 0.0203364   0.00213989   0.0315598   0.00596757  …   0.00160488  -0.00130837
 1.0         1.0          1.0         1.0             1.0          1.0
 1.0         1.0          1.0        -1.0             1.0         -1.0
 1.0         1.0          1.0         1.0            -1.0          1.0
 1.0        -1.0         -1.0         1.0            -1.0          1.0
 1.0         1.0         -1.0         1.0         …  -1.0         -1.0
 1.0         1.0         -1.0        -1.0            -1.0          1.0
 1.0         1.0         -1.0         1.0             1.0         -1.0
 1.0        -1.0          1.0         1.0             1.0         -1.0
 1.0         1.0         -1.0         1.0             1.0          1.0
 ⋮                                                ⋱                ⋮
 1.0        -1.0          1.0         1.0             1.0         -1.0
 1.0         1.0         -1.0         1.0             1.0          1.0
 1.0         1.0         -1.0        -1.0      

In [15]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R4 * x .== Cn)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.560660171779821
Optimal value of x: [0.01723942500485072, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.2829312048849847e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.2652193031436435e-16, 0.0, 0.0, 0.015625000000000333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0040212392637617575, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -3.279202433407018e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.018735264022772858, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004339284570099661, 0.0, 0.0, 0.0, -3.949871011820959e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [20]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R4 * x .== Kn)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.799936867076458
Optimal value of x: [0.0, -0.018295632294652895, 0.0025214605842464283, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0208168136188298, 0.003246807309209558, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -5.799596571460065e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.015527856871941425, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.010154038008676907, 0.0, 0.01012998488689077, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.017829904939787217, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014988602490141459, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -8.107614113911117e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008152716279509118, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.02249919614436666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

### less symmetric graphs

In [26]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R4 * x .== G1n)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 1.8624368670764562
Optimal value of x: [0.0, 0.0, 0.0, 0.0, 0.005959847987721632, 0.0, 0.004081475962597814, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008924309704051059, 0.0, 0.0, -4.4455125288892254e-17, 0.013832392528688876, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01638712303452184, 0.0, 0.0, 0.0, 0.0010671856341249047, 0.0006413388936527339, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.006247956089722571, 0.0, 0.0, 0.02149251953076833, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.01306242868025226, 0.016368675555655236, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022611240750278275, 0.0, -0.02790209633524235, 0.0, 0.0, 0.003280497904028249, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0015947237340864311, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014937411248132955, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008949847317389752, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005

In [27]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R4 * x .== G2n)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 2.112436867076456
Optimal value of x: [0.0, 0.0, 0.026543768471260217, 0.0, 0.0, 0.0, 0.008579252061268404, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.007399907505286316, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0004827876230865408, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.004048026026420315, 0.0, 0.0, 0.0, 0.0, 0.0, 5.3631086867683226e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -7.051355918292273e-19, 0.0, 1.6734856993464685e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.526538373610942e-16, 0.0, 0.0, 0.022955840704011698, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.008608060177732435, 0.0, 0.0024371415764036517, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [28]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R4)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R4 * x .== G3n)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

Optimal objective value (min ||x||_1): 2.010327766287467
Optimal value of x: [0.03168924676769437, 0.0, 0.0, 0.0, 0.0, 2.804242564144521e-16, 0.0, 0.0, 0.0, 0.026332945969957583, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.008677350800067483, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.00018897828300917896, 0.0, 0.0, 0.033364612236893805, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014430616132162635, 0.0, 0.0, 0.0, 0.0, 5.208553021838941e-17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.014537611619415936, 0.0, -3.625279895318536e-33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.690293646836911e-16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -6.688991456787783e-17, 0.0, 0.0, 0.0, 0.007598392044728733, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.007172557552785057, 0.0, 0.0, 

## Five-qubit LC robustness

In [45]:
using HDF5

# Load the HDF5 file
file = h5open("./data/deterministic_5.h5", "r")

# Assign the dataset to R5
R5 = file["deterministic_5"][:,:]

# Close the file
close(file)

# Use the matrix
println("Loaded matrix size: ", size(R5))

Loaded matrix size: (1024, 32768)


In [47]:
# four qubit magic state:
n = 5
Tn = t_state(n);

# line graph:
Ln = copy(Tn)
for i in 1:n-1
    Ln = cz_action_on_pauli_basis(i,i+1,Ln,pauli_dicts[n])
end

Cn = copy(Tn)
for i in 1:n
    if i < n
        Cn = cz_action_on_pauli_basis(i,i+1,Cn,pauli_dicts[n])
    else
        Cn = cz_action_on_pauli_basis(1,n,Cn,pauli_dicts[n])
    end
end

using Combinatorics

# Generate unique pairs (combinations) without self-loops
pairs = [p for p in combinations(1:n, 2)]

Kn = copy(Tn)
for p in pairs
    Kn = cz_action_on_pauli_basis(p[1],p[2],Kn,pauli_dicts[n])
end

In [ ]:

# Create a model with GLPK as the solver
model = Model(GLPK.Optimizer)
N = size(R5)[2];

# Define the decision variables
@variable(model, x[1:N])
@variable(model, u[1:N] >= 0)  # Auxiliary variables for the absolute values

# Objective: Minimize the sum of u (which represents |x|)
@objective(model, Min, sum(u))

# Constraints for the absolute values
@constraint(model, [i=1:N], u[i] >= x[i])
@constraint(model, [i=1:N], u[i] >= -x[i])

# Equality constraint: M * x = b
@constraint(model, R5 * x .== Kn)

# Solve the model
optimize!(model)

# Get the results
x_value = value.(x)
obj_value = objective_value(model)  # Use a different variable name

# Print the results
println("Optimal objective value (min ||x||_1): ", obj_value)
println("Optimal value of x: ", x_value)

In [93]:
using HDF5

# Load the HDF5 file
file = h5open("./deterministic_5_L5.h5", "r")

# Assign the dataset to R5
RL5 = file["deterministic_5_L5"][:,:]

# Close the file
close(file)

# Use the matrix
println("Loaded matrix size: ", size(RL5))

Loaded matrix size: (1025, 1050)
